#Import Data

In [5]:
!git clone -l -s git://github.com/AshraqatM/NLP_clustering.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), 1.44 MiB | 3.35 MiB/s, done.
/content/cloned-repo/cloned-repo
oversampling_dataset.csv


In [6]:
import pandas as pd
import numpy as np
df = pd.read_csv("oversampling_dataset.csv")
df.head()
df['category'] = df['category'].replace(['none','damning', 'derailing', 'discredit', 'dominance' , 'sexual harassment' , 'stereotyping & objectification' , 'threat of violence'],[0,1,2,3,4,5,6,7]) 
df = df[['text_normal_tokens','category']]

#Install transformers

In [2]:
# install simpletransformers
!pip install simpletransformers

# check installed version
!pip freeze | grep simpletransformers
# simpletransformers==0.28.2

     |████████████████████████████████| 221 kB 7.0 MB/s 
     |████████████████████████████████| 1.2 MB 14.0 MB/s 
     |████████████████████████████████| 3.3 MB 39.5 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 1.8 MB 59.7 MB/s 
     |████████████████████████████████| 2.6 MB 57.4 MB/s 
     |████████████████████████████████| 7.9 MB 61.7 MB/s 
     |████████████████████████████████| 542 kB 56.0 MB/s 
     |████████████████████████████████| 124 kB 33.5 MB/s 
     |████████████████████████████████| 76 kB 5.7 MB/s 
     |████████████████████████████████| 636 kB 54.2 MB/s 
     |████████████████████████████████| 895 kB 50.5 MB/s 
     |████████████████████████████████| 133 kB 59.0 MB/s 
     |████████████████████████████████| 170 kB 59.9 MB/s 
     |████████████████████████████████| 97 kB 8.7 MB/s 
     |████████████████████████████████| 138 kB 56.8 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |█████████████████

simpletransformers==0.61.10


#spliting

In [7]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.30)

print('train shape: ',train_df.shape)
print('test shape: ',test_df.shape)
train_df

train shape:  (14862, 2)
test shape:  (6370, 2)


,text_normal_tokens,category
18496,"['العالم', 'ام', 'انطمي', 'بتقلب', 'بيطيقك', '...",6
10020,"['اكلت', 'القتله', 'الناس', 'اله', 'اياه', 'با...",2
772,"['اكتر', 'الك', 'امك', 'بفيدك', 'تفتحي', 'خلي'...",0
10791,"['اخلاقك', 'بتفهمي', 'جد', 'كمخه', 'مسخره', 'ه...",3
17766,"['اكتر', 'بتمثلي', 'بحكيك', 'بهالخلقه', 'تضربي...",6
...,...,...
13169,"['السيد', 'الضاحيه', 'اهل', 'بقلبك', 'تاج', 'د...",4
12904,"['بتطق', 'بعضها', 'تحت', 'رقبتك', 'طويلي', 'قه...",4
8944,"['الاحرف', 'الصباط', 'بالاجر', 'بدو', 'بركي', ...",2
16271,"['المحن', 'ثوار', 'زعره', 'علنسوان', 'قوموا', ...",5


In [8]:
# 2-layer, 768-hidden, 12-heads, 179M parameters. Trained on cased text in the top 104 languages with the largest Wikipedias
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 2}

# Create a ClassificationModel
model = ClassificationModel(
    "xlm", "xlm-mlm-100-1280",
    num_labels=8,
    args=train_args
)

Downloading:   0%|          | 0.00/41.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-mlm-100-1280 were not used when initializing XLMForSequenceClassification: ['pred_layer.proj.bias', 'pred_layer.proj.weight']
- This IS expected if you are initializing XLMForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMForSequenceClassification were not initialized from the model checkpoint at xlm-mlm-100-1280 and are newly initialized: ['transformer.position_ids', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inf

Downloading:   0%|          | 0.00/5.72M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.97M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

In [9]:
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:616: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/14862 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/1858 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/1858 [00:00<?, ?it/s]

(3716, 0.4178174611255899)

In [10]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

result

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1402: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/6370 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/797 [00:00<?, ?it/s]

{'acc': 0.9566718995290424,
 'eval_loss': 0.2099973147198053,
 'f1': 0.9566718995290424,
 'mcc': 0.9504976171642273}

[]